<a href="https://colab.research.google.com/github/VictoriaCar/BA820---College-Scorecard-/blob/main/Brendan_Imputation_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brendan Imputation Testing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Cleaning

### Load and Inspect Data

In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/BA820 Project/MERGED2022_23_PP.csv")
# df = pd.read_csv("MERGED2022_23_PP.csv")

<ipython-input-2-31a2283f9ea8>:3: DtypeWarning: Columns (9,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,3215,3231,3235,3236) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/BA820 Project/MERGED2022_23_PP.csv")


In [3]:
df.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE0_P11,COUNT_WNE_MALE1_P11,GT_THRESHOLD_P11,MD_EARN_WNE_INC1_P11,MD_EARN_WNE_INC2_P11,MD_EARN_WNE_INC3_P11,MD_EARN_WNE_INDEP0_P11,MD_EARN_WNE_INDEP1_P11,MD_EARN_WNE_MALE0_P11,MD_EARN_WNE_MALE1_P11
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,https://www.amridgeuniversity.edu/,https://www2.amridgeuniversity.edu:9091/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(6484, 3305)

### Drop Columns at Different "Null Thresholds"

In [5]:
# If a column has more than 50% missing values --> drop it
missing_values = df.isnull().sum() / len(df) * 100
cols_missing = missing_values[missing_values > 50]
dropped_df = df.drop(columns = cols_missing.index)

print(f"Columns dropped: {len(cols_missing)}")
print(f"New DataFrame shape: {dropped_df.shape}")

Columns dropped: 2355
New DataFrame shape: (6484, 950)


### New ChatGPT-generated columns. No categoricals

In [8]:
# Selected columns (with specified ones removed)
selected_columns = [
    # Institutional Characteristics
    'PREDDEG',  # Predominant degree awarded
    'HIGHDEG',  # Highest degree awarded
    'CONTROL',  # Type of institution control
    'LOCALE',  # Degree of urbanization
    'CCBASIC',  # Carnegie Classification - Basic
    'CCUGPROF',  # Carnegie Classification - Undergraduate instructional program
    'CCSIZSET',  # Carnegie Classification - Size and setting
    'HBCU',  # Historically Black Colleges and Universities (1 = Yes, 0 = No)
    'HSI',  # Hispanic-Serving Institutions (1 = Yes, 0 = No)
    'DISTANCEONLY',  # Online-only institution (1 = Yes, 0 = No)

    # Location & Region
    'LATITUDE',  # Institution Latitude
    'LONGITUDE',  # Institution Longitude
    'ST_FIPS',  # State FIPS code
    'REGION',  # Geographic region

    # Student Demographics
    'UGDS',  # Total undergraduate student enrollment
    'PPTUG_EF',  # Part-time undergraduate enrollment share
    'UGDS_WHITE',  # White students' share of enrollment
    'UGDS_BLACK',  # Black students' share of enrollment
    'UGDS_HISP',  # Hispanic students' share of enrollment
    'UGDS_ASIAN',  # Asian students' share of enrollment
    'UGDS_AIAN',  # American Indian/Alaska Native share of enrollment
    'UGDS_NHPI',  # Native Hawaiian/Pacific Islander share of enrollment
    'UGDS_2MOR',  # Two or more races share of enrollment
    'UGDS_NRA',  # Non-resident aliens (international students)

    # Academic Offerings
    'PCIP14',  # Percentage of degrees in Engineering
    'PCIP11',  # Percentage of degrees in Computer Science
    'PCIP52',  # Percentage of degrees in Business, Marketing
    'PCIP51',  # Percentage of degrees in Health Professions
    'PCIP42',  # Percentage of degrees in Psychology
    'PCIP26',  # Percentage of degrees in Biological Sciences
    'PCIP45',  # Percentage of degrees in Social Sciences

    # Financials
    'NPT4_PRIV',  # Average net price for private institutions
    'NPT41_PRIV',  # Net price for $0-$30K family income (private)
    'NUM4_PRIV',  # Number of Title IV students (private institutions)

    # Outcomes
    'OMAWDP8_FTFT',  # Outcome metric for award pooling
    'CURROPER'  # School currently operating (1 = Yes, 0 = No)
]

# Filter the dataframe to only include the selected columns
filtered_df = dropped_df[selected_columns]

# Print out the selected column names and shape of the new dataframe
print(filtered_df.columns.tolist())
print(f"New DataFrame shape: {filtered_df.shape}")

['PREDDEG', 'HIGHDEG', 'CONTROL', 'LOCALE', 'CCBASIC', 'CCUGPROF', 'CCSIZSET', 'HBCU', 'HSI', 'DISTANCEONLY', 'LATITUDE', 'LONGITUDE', 'ST_FIPS', 'REGION', 'UGDS', 'PPTUG_EF', 'UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR', 'UGDS_NRA', 'PCIP14', 'PCIP11', 'PCIP52', 'PCIP51', 'PCIP42', 'PCIP26', 'PCIP45', 'NPT4_PRIV', 'NPT41_PRIV', 'NUM4_PRIV', 'OMAWDP8_FTFT', 'CURROPER']
New DataFrame shape: (6484, 36)


In [9]:
filtered_df.isnull().sum()

,0
PREDDEG,0
HIGHDEG,0
CONTROL,0
LOCALE,495
CCBASIC,495
CCUGPROF,495
CCSIZSET,495
HBCU,498
HSI,498
DISTANCEONLY,495


In [10]:
# Drop the specified columns from filtered_df
columns_to_drop = ['NPT4_PRIV', 'NPT41_PRIV', 'NUM4_PRIV', 'OMAWDP8_FTFT']

filtered_df = filtered_df.drop(columns=columns_to_drop, errors='ignore')

# Print updated column names and DataFrame shape
print(filtered_df.columns.tolist())
print(f"New DataFrame shape: {filtered_df.shape}")

['PREDDEG', 'HIGHDEG', 'CONTROL', 'LOCALE', 'CCBASIC', 'CCUGPROF', 'CCSIZSET', 'HBCU', 'HSI', 'DISTANCEONLY', 'LATITUDE', 'LONGITUDE', 'ST_FIPS', 'REGION', 'UGDS', 'PPTUG_EF', 'UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR', 'UGDS_NRA', 'PCIP14', 'PCIP11', 'PCIP52', 'PCIP51', 'PCIP42', 'PCIP26', 'PCIP45', 'CURROPER']
New DataFrame shape: (6484, 32)


In [11]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6484 entries, 0 to 6483
Data columns (total 32 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PREDDEG       6484 non-null   int64  
 1   HIGHDEG       6484 non-null   int64  
 2   CONTROL       6484 non-null   int64  
 3   LOCALE        5989 non-null   float64
 4   CCBASIC       5989 non-null   float64
 5   CCUGPROF      5989 non-null   float64
 6   CCSIZSET      5989 non-null   float64
 7   HBCU          5986 non-null   float64
 8   HSI           5986 non-null   float64
 9   DISTANCEONLY  5989 non-null   float64
 10  LATITUDE      5989 non-null   float64
 11  LONGITUDE     5989 non-null   float64
 12  ST_FIPS       6484 non-null   int64  
 13  REGION        6484 non-null   int64  
 14  UGDS          5716 non-null   float64
 15  PPTUG_EF      5688 non-null   float64
 16  UGDS_WHITE    5716 non-null   float64
 17  UGDS_BLACK    5716 non-null   float64
 18  UGDS_HISP     5716 non-null 

In [12]:
filtered_df.head()

,PREDDEG,HIGHDEG,CONTROL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,HBCU,HSI,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,PCIP14,PCIP11,PCIP52,PCIP51,PCIP42,PCIP26,PCIP45,CURROPER
0,3,4,1,12.0,18.0,10.0,14.0,1.0,0.0,0.0,...,0.0127,0.0115,0.1076,0.0802,0.1526,0.0000,0.0744,0.1644,0.0313,1
1,3,4,1,12.0,15.0,9.0,15.0,0.0,0.0,0.0,...,0.0491,0.0237,0.0600,0.0241,0.1934,0.2213,0.0792,0.1250,0.0265,1
2,3,4,2,12.0,20.0,5.0,6.0,0.0,0.0,1.0,...,0.0000,0.0000,0.0000,0.0000,0.4565,0.0000,0.0000,0.0000,0.0000,1
3,3,4,1,12.0,15.0,15.0,13.0,0.0,0.0,0.0,...,0.0465,0.0149,0.2924,0.0906,0.1945,0.1119,0.0249,0.0553,0.0249,1
4,3,4,1,12.0,17.0,10.0,14.0,1.0,0.0,0.0,...,0.0118,0.0221,0.0104,0.0750,0.1500,0.0500,0.0521,0.1271,0.0271,1


* after dropping NPT4_PRIV, NPT41_PRIV, NUM4_PRIV, OMAWDP8_FTFT, column most nulls in a column is 773 (~11% of obs).
* Keep demographic columns because numerical and impuation should be accurate *enough*

### KNN Imputation

In [13]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Standardize the data before KNN (KNN works better with scaled values)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(filtered_df)

# Apply KNN Imputation
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")  # n_neighbors=5 is default
imputed_data = knn_imputer.fit_transform(scaled_data)

# Convert back to original scale
imputed_df = pd.DataFrame(scaler.inverse_transform(imputed_data), columns=filtered_df.columns)

# Print summary
print("KNN Imputation Completed.")
print(f"Updated DataFrame shape: {imputed_df.shape}")
print(imputed_df.isnull().sum())  # Verify no missing values

KNN Imputation Completed.
Updated DataFrame shape: (6484, 32)
PREDDEG         0
HIGHDEG         0
CONTROL         0
LOCALE          0
CCBASIC         0
CCUGPROF        0
CCSIZSET        0
HBCU            0
HSI             0
DISTANCEONLY    0
LATITUDE        0
LONGITUDE       0
ST_FIPS         0
REGION          0
UGDS            0
PPTUG_EF        0
UGDS_WHITE      0
UGDS_BLACK      0
UGDS_HISP       0
UGDS_ASIAN      0
UGDS_AIAN       0
UGDS_NHPI       0
UGDS_2MOR       0
UGDS_NRA        0
PCIP14          0
PCIP11          0
PCIP52          0
PCIP51          0
PCIP42          0
PCIP26          0
PCIP45          0
CURROPER        0
dtype: int64


In [14]:
imputed_df.head()

,PREDDEG,HIGHDEG,CONTROL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,HBCU,HSI,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,PCIP14,PCIP11,PCIP52,PCIP51,PCIP42,PCIP26,PCIP45,CURROPER
0,3.0,4.0,1.0,12.0,18.0,10.0,14.0,1.0,0.0,0.0,...,0.0127,0.0115,0.1076,0.0802,0.1526,0.0000,0.0744,0.1644,0.0313,1.0
1,3.0,4.0,1.0,12.0,15.0,9.0,15.0,0.0,0.0,0.0,...,0.0491,0.0237,0.0600,0.0241,0.1934,0.2213,0.0792,0.1250,0.0265,1.0
2,3.0,4.0,2.0,12.0,20.0,5.0,6.0,0.0,0.0,1.0,...,0.0000,0.0000,0.0000,0.0000,0.4565,0.0000,0.0000,0.0000,0.0000,1.0
3,3.0,4.0,1.0,12.0,15.0,15.0,13.0,0.0,0.0,0.0,...,0.0465,0.0149,0.2924,0.0906,0.1945,0.1119,0.0249,0.0553,0.0249,1.0
4,3.0,4.0,1.0,12.0,17.0,10.0,14.0,1.0,0.0,0.0,...,0.0118,0.0221,0.0104,0.0750,0.1500,0.0500,0.0521,0.1271,0.0271,1.0


In [15]:
imputed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6484 entries, 0 to 6483
Data columns (total 32 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PREDDEG       6484 non-null   float64
 1   HIGHDEG       6484 non-null   float64
 2   CONTROL       6484 non-null   float64
 3   LOCALE        6484 non-null   float64
 4   CCBASIC       6484 non-null   float64
 5   CCUGPROF      6484 non-null   float64
 6   CCSIZSET      6484 non-null   float64
 7   HBCU          6484 non-null   float64
 8   HSI           6484 non-null   float64
 9   DISTANCEONLY  6484 non-null   float64
 10  LATITUDE      6484 non-null   float64
 11  LONGITUDE     6484 non-null   float64
 12  ST_FIPS       6484 non-null   float64
 13  REGION        6484 non-null   float64
 14  UGDS          6484 non-null   float64
 15  PPTUG_EF      6484 non-null   float64
 16  UGDS_WHITE    6484 non-null   float64
 17  UGDS_BLACK    6484 non-null   float64
 18  UGDS_HISP     6484 non-null 